In [1]:
import cPickle as pickle
import numpy as np

In [2]:
import numpy as np
import os
import pickle

from sklearn.model_selection 		 import train_test_split
from keras.applications.vgg16 		 import VGG16
from keras.preprocessing.image       import ImageDataGenerator
from keras.models                    import Sequential, Model, Input
from keras.layers                    import Dense, Flatten, Dropout
from keras.optimizers                import SGD, RMSprop, Adam
from keras.callbacks                 import EarlyStopping
from keras.utils                     import np_utils
from keras                           import backend as K

def run(X, Y, pickle_filename, model_filename, batch_size=32, num_epochs=50): 

	# Split into test and train/validation set
	X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42, stratify=Y)

	# Split into train and validation
	X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, train_size=0.9, random_state=42, stratify=y_train) 
	print('Train shape: ', X_train.shape, y_train.shape)
	print('Val shape: ', X_val.shape, y_val.shape)
	print('Test shape: ', X_test.shape, np.array(y_test).shape)

	# Categorize the labels
	num_classes = 2
	y_train = np_utils.to_categorical(y_train, num_classes)
	y_val = np_utils.to_categorical(y_val, num_classes)
	y_test = np_utils.to_categorical(y_test, num_classes)
	print("y_train, y_val, y_test: ", y_train.shape, y_val.shape, y_test.shape)

	# Create the base pre-trained model
	base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

	x = base_model.output
	x = Flatten()(x)
	x = Dense(2048, activation='relu')(x)
	x = Dropout(.7)(x)
	x = Dense(2048, activation='relu')(x)
	predictions = Dense(num_classes, activation='softmax')(x)

	model = Model(inputs=base_model.input, outputs=predictions)
	# print(model.summary())

	k = 12 # number of end layers to retrain CHANGE THIS ALISTAIR SAID TO RETRAIN THE LAST CNN5(?) idk how many layers that is (8+4)
	layers = base_model.layers[:-k] if k != 0 else base_model.layers
	for layer in layers: 
	    layer.trainable = False

	# Compile model
	opt = SGD(lr=0.0001, momentum=0.9)
	model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

	# Initiate the train, validation and test generators with data augumentation
	train_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True, vertical_flip = True)
	train_datagen.fit(X_train)
	generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

	val_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True, vertical_flip = True)
	val_datagen.fit(X_val)
	val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

	test_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True, vertical_flip = True)
	test_datagen.fit(X_test)
	test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)


	# Train the model, auto terminating when val_acc stops increasing after 10 epochs.
	# callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=2, mode='max') 
    # , callbacks=[callback],
	hist = model.fit_generator(generator, steps_per_epoch=len(X_train) / batch_size , epochs=num_epochs, verbose=1, validation_data=val_generator, validation_steps=len(X_val)/batch_size)

	# Save accuracy / loss during training to pickle file so we can plot later
	pickle.dump(hist.history, open(pickle_filename, 'wb'))

	# Evalulate model
	test_loss, accuracy = model.evaluate_generator(test_generator, X_test.shape[0])
	print('Test loss: ', test_loss, ' Accuracy: ', accuracy)

	# Save model
	model.save(model_filename)


Using TensorFlow backend.


In [3]:
x_True = pickle.load(open( "x_true.p", "rb" ))
x_False = pickle.load(open( "x_neg.p", "rb" ))

In [4]:
y_True = [1]*len(x_True)
y_False = [0]*len(x_False)

In [5]:
X = np.concatenate((x_True, x_False))
Y = np.concatenate((y_True, y_False))
print X.shape, Y.shape

(5471, 224, 224, 3) (5471,)


In [6]:
run(X, Y, 'full_hist.pkl', 'full_model.h5', batch_size=32, num_epochs=200)

/home/alyssayc/.virtualenvs/venv/local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


('Train shape: ', (3938, 224, 224, 3), (3938,))
('Val shape: ', (438, 224, 224, 3), (438,))
('Test shape: ', (1095, 224, 224, 3), (1095,))
('y_train, y_val, y_test: ', (3938, 2), (438, 2), (1095, 2))
Epoch 1/200
123/123 [==============================] - 26s 214ms/step - loss: 0.7672 - acc: 0.4983 - val_loss: 0.7649 - val_acc: 0.4495
Epoch 2/200
123/123 [==============================] - 26s 208ms/step - loss: 0.7210 - acc: 0.5297 - val_loss: 0.7013 - val_acc: 0.4495
Epoch 3/200
123/123 [==============================] - 25s 206ms/step - loss: 0.7084 - acc: 0.5107 - val_loss: 0.6959 - val_acc: 0.4495
Epoch 4/200
123/123 [==============================] - 26s 208ms/step - loss: 0.6948 - acc: 0.5388 - val_loss: 0.6933 - val_acc: 0.4495
Epoch 5/200
123/123 [==============================] - 25s 205ms/step - loss: 0.6959 - acc: 0.5269 - val_loss: 0.6900 - val_acc: 0.5505
Epoch 6/200
123/123 [==============================] - 25s 206ms/step - loss: 0.6899 - acc: 0.5427 - val_loss: 0.6939 - 

123/123 [==============================] - 24s 199ms/step - loss: 0.6865 - acc: 0.5556 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 60/200
123/123 [==============================] - 25s 202ms/step - loss: 0.6885 - acc: 0.5534 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 61/200
123/123 [==============================] - 25s 203ms/step - loss: 0.6886 - acc: 0.5511 - val_loss: 0.6883 - val_acc: 0.5505
Epoch 62/200
123/123 [==============================] - 25s 201ms/step - loss: 0.6883 - acc: 0.5524 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 63/200
123/123 [==============================] - 25s 203ms/step - loss: 0.6882 - acc: 0.5516 - val_loss: 0.6883 - val_acc: 0.5505
Epoch 64/200
123/123 [==============================] - 25s 203ms/step - loss: 0.6862 - acc: 0.5615 - val_loss: 0.6880 - val_acc: 0.5505
Epoch 65/200
123/123 [==============================] - 25s 200ms/step - loss: 0.6905 - acc: 0.5409 - val_loss: 0.6887 - val_acc: 0.5505
Epoch 66/200
123/123 [==============================] 

Epoch 119/200
123/123 [==============================] - 24s 198ms/step - loss: 0.6882 - acc: 0.5521 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 120/200
123/123 [==============================] - 24s 196ms/step - loss: 0.6885 - acc: 0.5514 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 121/200
123/123 [==============================] - 24s 195ms/step - loss: 0.6887 - acc: 0.5481 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 122/200
123/123 [==============================] - 24s 195ms/step - loss: 0.6892 - acc: 0.5468 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 123/200
123/123 [==============================] - 24s 194ms/step - loss: 0.6881 - acc: 0.5514 - val_loss: 0.6882 - val_acc: 0.5505
Epoch 124/200
123/123 [==============================] - 24s 193ms/step - loss: 0.6881 - acc: 0.5531 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 125/200
123/123 [==============================] - 24s 194ms/step - loss: 0.6878 - acc: 0.5528 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 126/200
123/123 [===========

123/123 [==============================] - 25s 202ms/step - loss: 0.6885 - acc: 0.5483 - val_loss: 0.6880 - val_acc: 0.5505
Epoch 179/200
123/123 [==============================] - 25s 204ms/step - loss: 0.6882 - acc: 0.5493 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 180/200
123/123 [==============================] - 25s 202ms/step - loss: 0.6871 - acc: 0.5567 - val_loss: 0.6880 - val_acc: 0.5505
Epoch 181/200
123/123 [==============================] - 25s 204ms/step - loss: 0.6841 - acc: 0.5668 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 182/200
123/123 [==============================] - 25s 200ms/step - loss: 0.6905 - acc: 0.5422 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 183/200
123/123 [==============================] - 25s 202ms/step - loss: 0.6857 - acc: 0.5610 - val_loss: 0.6880 - val_acc: 0.5505
Epoch 184/200
123/123 [==============================] - 25s 200ms/step - loss: 0.6906 - acc: 0.5391 - val_loss: 0.6881 - val_acc: 0.5505
Epoch 185/200
123/123 [=========================